In [1]:
from json2csv import load_measure_annotation

In [2]:
img_path = './tmp/data_20'
json_path = './tmp/data_20_json'
anno_csv_path = './tmp/val_annots_div.csv'

In [3]:
load_measure_annotation(img_path, json_path, anno_csv_path)
print("Finished!")

Finished!


In [4]:
import torch
from csv_eval import evaluate

In [5]:
model_path = './csv/csv_retinanet_alldiv_best.pth'
retinanet = torch.load(model_path)

In [6]:
retinanet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [8]:
import time
import os
import copy
import argparse
import pdb
import collections
import sys

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torchvision

import model
from anchors import Anchors
from utils import _transfer_pretrained_weights
import losses

from dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer
from torch.utils.data import Dataset, DataLoader

In [12]:
dataset_val = CSVDataset(train_file=anno_csv_path, class_list='./csv/class_list_div.csv', transform=transforms.Compose([Normalizer(), Resizer()]))

In [13]:
sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=3, collate_fn=collater, batch_sampler=sampler_val)

In [14]:
APs, mAP = evaluate(dataset_val, retinanet)

19/19
label_AP:
div: 0.8344296490107336
mAP: 0.8344296490107336
